<a href="https://colab.research.google.com/github/ti-lei/data-course-sample/blob/main/%E3%80%8Csample_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
# test
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [3]:
# windows (!python -m)跟 在 colab(!wget) 有差
!python -m wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!python -m wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz


Saved under All_Beauty.csv

Saved under meta_All_Beauty.json.gz


In [331]:
metadata = getDF('meta_All_Beauty.json.gz')
# 原始資料沒有 header 另外命名給他
ratings = pd.read_csv('All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)
# 輸出metada 到csv
# metadata.to_csv('metada.csv',index=False)

In [332]:
ratings.head()
# 將 metada 取出要用的資訊
maindata = metadata[['rank','price','asin']]
# 直接把重複商品的資料丟掉
maindata = maindata.drop_duplicates(subset=['asin'])

## 資料整理

In [333]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
# 把 unixReviewTime drop
ratings.drop(columns=['unixReviewTime'],inplace = True)

## 資料切分

In [334]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
# agg(list)會把所有除了reviewerID的欄位用陣列包住
# to_dict 會把 dataframe 轉成字典(一列一個字典) 並用 陣列包住
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')

# 把 [{'asin': ['B01DKQAXC0'], 'reviewerID': 'A100XQFWKQ30O2'},..] 變成 {A100XQFWKQ30O2:['B01DKQAXC0'],...}
# key: reviwerID value: asin
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }

# users:[user1,user2....]
users = list(ratings_testings_by_user.keys())

# 應該要測試 users 在測試的資料裡 是否有跟 訓練的資料重疊 如果沒有的話根本不知道其消費行為 根本隨便推薦:


# 驗證 Training data 的消費者有沒有在 Training data 出現

In [335]:

training_users = ratings_trainings['reviewerID'].drop_duplicates().to_list()
overlap = 0
for user in users:
    if user in training_users:
        overlap += 1
overlap

38

# maindata資料整理

In [336]:
from datetime import datetime
from dateutil.relativedelta import *

# 紀錄每個時間間格購買數量
buy_in_every_month = pd.DataFrame()
#起始時間
basic_date = datetime(2018, 3, 1)
# 預計要推薦的商品個數
k = 10
for i in range(6):
    # 開始時間
    start_date = basic_date+relativedelta(months=+i)
    # 結束時間
    end_date = basic_date+relativedelta(months=+i+1)

    buy_last_one_month = ratings_trainings[
        (ratings['DATE'] < end_date.strftime("%Y-%m-%d")) & (ratings['DATE'] > start_date.strftime("%Y-%m-%d"))
    ]
    buy_last_one_month = buy_last_one_month.groupby(by=['asin']).size()
    #數據加權 越接近資料時間的終點加權越重
    weight = i + 1
    buy_last_one_month = weight * buy_last_one_month
    #  第一次不能用 inner join 不然dataframe是空的
    if i== 0:
        buy_in_every_month = pd.concat([buy_in_every_month, buy_last_one_month])
    # axis = 1 是以欄為單位 往右邊去 合併
    else:
        buy_in_every_month = pd.concat([buy_in_every_month, buy_last_one_month], axis=1)
buy_in_every_month.columns = ['a','b','c','d','e','f']
#第一層濾網先把前20名銷量篩出來
top50 = buy_in_every_month.sum(axis=1).sort_values(ascending = False)[:50].index
#第二層將前三個月完全沒有銷量的產品篩掉
target = buy_in_every_month.loc[top50,:].dropna(subset=['d', 'e','f']).index[:k]
buy_in_every_month.loc[target,:]

<ipython-input-336-574165ba11ea>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buy_last_one_month = ratings_trainings[


,a,b,c,d,e,f
B01DKQAXC0,193.0,262.0,375.0,428.0,520.0,438.0
B00W259T7G,50.0,76.0,138.0,232.0,260.0,294.0
B0195R1FT8,84.0,144.0,156.0,72.0,100.0,138.0
B012Z7IHHI,19.0,50.0,78.0,112.0,100.0,150.0
B013XKHA4M,15.0,28.0,36.0,76.0,85.0,168.0
B01AVJCDYA,4.0,10.0,36.0,60.0,100.0,132.0
B00JVU3K9I,6.0,8.0,66.0,96.0,150.0,6.0
B018WCT01C,16.0,52.0,69.0,36.0,75.0,72.0
B0168SXRR0,7.0,20.0,39.0,56.0,85.0,72.0
B00X4DKZKU,5.0,26.0,36.0,60.0,70.0,60.0


In [338]:
buy_in_every_month.loc[top50,:].head(10)

,a,b,c,d,e,f
B01DKQAXC0,193.0,262.0,375.0,428.0,520.0,438.0
B00W259T7G,50.0,76.0,138.0,232.0,260.0,294.0
B0195R1FT8,84.0,144.0,156.0,72.0,100.0,138.0
B006IB5T4W,91.0,156.0,186.0,228.0,NaN,NaN
B012Z7IHHI,19.0,50.0,78.0,112.0,100.0,150.0
B013XKHA4M,15.0,28.0,36.0,76.0,85.0,168.0
B001QY8QXM,55.0,114.0,126.0,64.0,NaN,NaN
B0012Y0ZG2,36.0,164.0,144.0,NaN,NaN,NaN
B01AVJCDYA,4.0,10.0,36.0,60.0,100.0,132.0
B00JVU3K9I,6.0,8.0,66.0,96.0,150.0,6.0


## 產生推薦

In [339]:
def recommender(training_data, users=[], k=10):

    recommendations = {users[i]:target.to_list() for i in range(len(users))}
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)

## 結果評估

In [340]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 是測試資料裡 key:user value:商品ID 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
#         if d in ratings_by_user:
        total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
    print(total,len(ratings_testings))
    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

76 590


0.1288135593220339